Primeiramente importamos uns módulos útiles.

In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy             as sc
import time              as time
import os                as os
import matlab.engine

from scipy                import special
from mpl_toolkits.mplot3d import Axes3D
from matplotlib           import path
from matplotlib.path      import Path
from tqdm                 import tqdm


t0 = time.perf_counter()

Inicializamos parámetros básicos.

In [2]:
boundary = 'Rectangle'
a = 12.5

max_in_sum_1 = 60 # Puntos para cada orde.
max_in_sum_2 = 60 # Ordes de F_n.

M = 1
N = 1

spacing_subarray = 0.5
spacing_dipoles  = 0.5

Decidimos se calculamos as táboas.

In [3]:
Element = False
Subarray = False

Definimos unha función para determinar puntos dentro e fóra da máscara e o seu raio.

In [4]:
def pass_to_dB(function):
    return 10 * np.log10(function / np.max(function))

def clear_around(function, limit):
    function[np.where(function < limit)] = limit
    #function[out_arg] = limit

def degrees(angle):
    return 180 * angle / np.pi

def radians(angle):
    return np.pi * angle / 180

def _ancho_(phi_rad_0):
    theta = 0
    
    for i in np.arange(1, 5): # Calculamos o ancho exacto a -0.50 dB.
        step = 10 / 8**i
        temp = 1
        
        while temp > 0:
            theta += step
            u = np.sin(radians(theta)) * np.cos(phi_rad_0)
            v = np.sin(radians(theta)) * np.sin(phi_rad_0)
            temp = mask_path.contains_point((u, v)) * 1 # 1 se o contén, 0 se non.
        
        theta -= step
    return np.sin(radians(theta))

def track_radio_mask(radio_0, sin_theta_0, mask, phi_rad):
    sin_theta_vector = np.zeros(len(phi_rad))
    
    for i in range(len(phi_rad)):
        sin_theta_vector[i] = _ancho_(phi_rad[i])
    
    return radio_0 * sin_theta_0 / sin_theta_vector

Procedemos ao cálculos coa máscara.

In [5]:
mask = np.loadtxt(f'Masks/mask_boundary_{boundary}.dat')
mask_path = Path((mask))

radio_0 = 50 # Radio de 50 lambda.
sin_theta_0 = 0.04273

phi_deg = np.arange(-180, 180 + 0.1, 0.1)
phi_rad = np.pi * phi_deg / 180
radio_vector = track_radio_mask(radio_0, sin_theta_0, mask, phi_rad) # Creamos o vector radio.


Gardamos o raio vector para acceder a el dende Matlab.

In [6]:
np.savetxt('Temporal_radial_vector.txt', np.column_stack(radio_vector) / radio_0)

## Zeros de Bessel

In [7]:
rows = 2 * max_in_sum_2 + 1
columns = max_in_sum_1 + 1

Bessel_zeros = np.zeros((max_in_sum_2 + 1, columns))

for k in range(max_in_sum_2 + 1):
    Bessel_zeros[k, 1:max_in_sum_1 + 1] = sc.special.jn_zeros(k, max_in_sum_1) / np.pi

## Táboas de $F_n(p)$

$F_n$ é o que se calcula en Matlab.

In [ ]:
txt_name = f'Results/Tables/Table_F_n_{boundary}_{max_in_sum_1}_{max_in_sum_2}.txt'
if os.path.exists(txt_name):
    F_k_loaded = np.loadtxt(txt_name, dtype='complex')
else:
    eng = matlab.engine.start_matlab()    
    F_k_loaded = np.zeros((rows, columns), dtype='complex')
    
    for k in tqdm(np.arange(- max_in_sum_2, max_in_sum_2 + 1)):        
        if k == 0:
            gamma_nm = Bessel_zeros[1, :]
        else:
            gamma_nm = Bessel_zeros[np.abs(k), :]
        
        for zero in range(columns):
            if gamma_nm[zero] in Bessel_zeros[1, :]:
                gamma_nm[zero] += 1e-5 # Cambiei esto de -15 a -5.
            
            try:
                F_k_loaded[k + max_in_sum_2, zero] = eng.Integral(float(gamma_nm[zero]), float(k)) # Función de Matlab
            except:
                eng.quit()
                eng = matlab.engine.start_matlab()
                F_k_loaded[k + max_in_sum_2, zero] = eng.Integral(float(gamma_nm[zero]), float(k)) # Función de Matlab
            
    np.savetxt(txt_name, F_k_loaded)
    eng.quit()

  0%|                                                                                                                                                                    | 0/121 [00:00<?, ?it/s]


valor =

   7.1125e-15 - 4.7211e-16i


valor =

  -0.0013 + 0.0000i


valor =

  -7.7591e-05 + 4.2443e-06i


valor =

   1.1772e-05 - 4.6236e-06i


valor =

   5.5206e-04 + 1.5914e-07i


valor =

  -3.6825e-04 + 5.0120e-05i


valor =

  -1.9392e-04 + 2.2438e-06i


valor =

   3.9020e-04 - 2.0062e-06i


valor =

  -1.7703e-04 + 1.3912e-06i


valor =

  -1.2391e-04 + 3.0505e-07i


valor =

   2.2220e-04 - 1.7046e-06i


valor =

  -1.1623e-04 + 2.3207e-06i


valor =

  -5.8444e-05 - 1.9373e-06i


valor =

   1.5071e-04 + 1.2047e-06i


valor =

  -1.0320e-04 - 1.0099e-06i


valor =

  -3.0407e-05 + 1.7211e-06i


valor =

   1.8850e-04 + 1.3656e-06i


valor =

  -1.3039e-04 + 7.3573e-08i


valor =

   4.6187e-06 - 2.2372e-06i


valor =

   1.2551e-04 + 2.7709e-07i


valor =

  -1.4018e-04 + 1.5020e-06i


valor =

  -4.1283e-05 - 2.0485e-06i


valor =

   1.5110e-04 + 1.2740e-06i


valor =

  -1.6352e-04 + 3.4088e-06i


valor =

  -4.4039e-06 - 6.2447e-07i


valor =

   1.5782e-04 + 3.3613e

  1%|█▎                                                                                                                                                          | 1/121 [00:05<10:27,  5.23s/it]


valor =

   2.4794e-05 + 2.6774e-06i


valor =

  -8.2267e-05 - 4.2142e-06i


valor =

   2.6975e-05 - 6.4006e-07i


valor =

   6.8832e-05 + 6.3034e-07i


valor =

   6.8067e-15 + 6.4936e-16i


valor =

   1.7090e-06 - 1.2302e-05i


valor =

   8.5951e-06 - 4.7295e-07i


valor =

   2.5695e-06 + 1.6662e-06i


valor =

   5.6468e-06 + 9.2033e-08i


valor =

  -4.9800e-06 - 1.3519e-06i


valor =

   2.5362e-06 - 3.0428e-07i


valor =

   9.9417e-06 + 6.3692e-06i


valor =

  -7.9341e-06 + 2.0311e-06i


valor =

  -1.0994e-05 + 5.5734e-08i


valor =

   1.0651e-05 + 4.8239e-06i


valor =

  -8.8187e-06 + 5.6860e-07i


valor =

   7.1185e-06 - 1.9478e-06i


valor =

  -1.0285e-06 + 2.1073e-06i


valor =

  -1.4920e-06 + 3.5140e-06i


valor =

   9.0466e-06 - 1.1010e-06i


valor =

  -1.3900e-05 - 1.2789e-05i


valor =

  -3.1346e-05 - 6.3505e-06i


valor =

   2.8588e-05 + 2.3601e-05i


valor =

  -1.0138e-05 + 1.2138e-06i


valor =

   3.0961e-06 - 8.5448e-07i


valor =

   6.5490e-06 -

  2%|██▌                                                                                                                                                         | 2/121 [00:06<05:23,  2.72s/it]

...


valor =

  -3.2813e-05 - 2.9332e-06i


valor =

  -5.6788e-05 - 3.5731e-07i


valor =

   3.6650e-05 + 2.1494e-06i


valor =

  -3.0745e-14 + 1.6731e-16i


valor =

  -0.0012 - 0.0000i


valor =

  -7.8785e-05 - 1.4295e-07i


valor =

   3.0647e-04 - 1.3029e-06i


valor =

   3.1806e-04 + 4.5498e-06i


valor =

  -3.2912e-04 - 4.5673e-06i


valor =

  -1.0416e-04 + 3.9341e-06i


valor =

   3.1938e-04 - 1.3623e-06i


valor =

  -1.3290e-04 + 5.4364e-07i


valor =

  -1.3801e-04 + 2.4466e-07i


valor =

   2.2718e-04 - 4.3849e-06i


valor =

  -6.9465e-05 + 4.5628e-06i


valor =

  -1.4467e-05 - 3.7375e-06i


valor =

   1.9118e-04 + 2.1647e-06i


valor =

  -5.2041e-05 + 7.4505e-07i


valor =

  -1.2720e-04 - 2.5753e-06i


valor =

   1.7730e-04 + 2.3482e-06i


valor =

  -4.6631e-05 - 4.3428e-06i


valor =

  -1.3366e-04 + 1.4715e-06i


valor =

   1.6496e-04 + 4.1276e-07i


valor =

  -5.2135e-05 + 1.5302e-06i


valor =

  -6.0966e-05 - 2.7950e-06i


valor =

   9.5931e-05 + 2.2999e

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍  | 119/121 [01:43<00:01,  1.14it/s]


valor =

   1.2303e-05 + 3.7081e-06i


valor =

  -2.1040e-04 - 4.0553e-06i


valor =

   1.1825e-04 + 2.1157e-05i


valor =

   2.6547e-05 + 1.6933e-06i


valor =

  -1.5635e-04 - 2.1313e-06i


valor =

   2.3831e-04 - 2.3289e-06i


valor =

   6.8067e-15 - 6.4936e-16i


valor =

   1.7090e-06 + 1.2302e-05i


valor =

   8.5951e-06 + 4.7295e-07i


valor =

   2.5695e-06 - 1.6662e-06i


valor =

   5.6468e-06 - 9.2033e-08i


valor =

  -4.9800e-06 + 1.3519e-06i


valor =

   2.5362e-06 + 3.0428e-07i


valor =

   9.9417e-06 - 6.3692e-06i


valor =

  -7.9341e-06 - 2.0311e-06i


valor =

  -1.0994e-05 - 5.5734e-08i


valor =

   1.0651e-05 - 4.8239e-06i


valor =

  -8.8187e-06 - 5.6860e-07i


valor =

   7.1185e-06 + 1.9478e-06i


valor =

  -1.0285e-06 - 2.1073e-06i


valor =

  -1.4920e-06 - 3.5140e-06i


valor =

   9.0466e-06 + 1.1010e-06i


valor =

  -1.3900e-05 + 1.2789e-05i


valor =

  -3.1346e-05 + 6.3505e-06i


valor =

   2.8588e-05 - 2.3601e-05i


valor =

  -1.0138e-05 -

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 120/121 [01:44<00:00,  1.13it/s]


valor =

   3.1170e-05 - 1.7615e-06i


valor =

  -4.1324e-06 - 1.2271e-06i


valor =

  -1.3664e-05 + 5.5314e-07i


valor =

  -4.0950e-05 - 8.4038e-08i


valor =

   4.9538e-05 - 4.3914e-06i


valor =

   3.0085e-07 + 2.7093e-06i


valor =

  -9.9339e-05 + 3.3683e-06i


valor =

   1.7922e-05 + 3.2338e-07i


valor =

   2.2586e-05 + 4.6170e-07i


valor =

  -1.1920e-04 + 1.6218e-06i


valor =

  -2.5590e-05 - 9.5625e-07i


valor =

   3.4517e-05 + 1.9886e-06i


valor =

   7.1125e-15 + 4.7211e-16i


valor =

  -0.0013 - 0.0000i


valor =

  -7.7591e-05 - 4.2443e-06i


valor =

   1.1772e-05 + 4.6236e-06i


valor =

   5.5206e-04 - 1.5914e-07i


valor =

  -3.6825e-04 - 5.0120e-05i


valor =

  -1.9392e-04 - 2.2438e-06i


valor =

   3.9020e-04 + 2.0062e-06i


valor =

  -1.7703e-04 - 1.3912e-06i


valor =

  -1.2391e-04 - 3.0505e-07i


valor =

   2.2220e-04 + 1.7046e-06i


valor =

  -1.1623e-04 - 2.3207e-06i


valor =

  -5.8444e-05 + 1.9373e-06i


valor =

   1.5071e-04 - 1.2047e

## Calculamos $g_n(p)$

In [ ]:
def g_n(k, rho): # Para todo n.
    aux = 0
    p = np.pi * rho / a
    
    if k == 0:
        for m in np.arange(0, max_in_sum_1):
            gamma_1m = Bessel_zeros[1, m]
            aux += (F_k_loaded[max_in_sum_2, m] * sc.special.jv(0, gamma_1m * p)) / (sc.special.jv(0, gamma_1m * np.pi))**2 
    
    else:
        for m in np.arange(1, max_in_sum_1):
            gamma_nm = Bessel_zeros[np.abs(k), m]
            aux -= (F_k_loaded[k + max_in_sum_2, m] * sc.special.jv(k, gamma_nm * p)) / (sc.special.jv(k - 1, gamma_nm * np.pi) * sc.special.jv(k + 1, gamma_nm * np.pi))
        
    return 2 * aux / np.pi**2

## Calculamos $K_n(\rho)$

In [ ]:
def K_n(k, p):
    return g_n(k, p) * np.pi / (2 * a**2 * (1j)**k)

## Táboas de $K_n(\rho)$

In [ ]:
radius_length = int((a + 0.5) / 0.01) + 1
radius = np.flip(np.linspace(a, 0, radius_length, endpoint = False))

txt_name = f'Results/Tables/Table_K_n_a_{a}_{boundary}_{max_in_sum_1}_{max_in_sum_2}.txt'
if os.path.exists(txt_name):
    K_n_loaded = np.loadtxt(txt_name, dtype='complex')
else:
    K_n_loaded = np.zeros((2 * max_in_sum_2 + 1, radius_length), dtype='complex')

    for k in tqdm(np.arange(-max_in_sum_2, max_in_sum_2 + 1)):
        for j in range(radius_length):
            K_n_loaded[k + max_in_sum_2, j] = K_n(k, radius[j])

    np.savetxt(txt_name, K_n_loaded)

## Calculamos $K(\rho, \beta)$

In [ ]:
def find_nearest(array, value): # It returns the position.
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return int(idx)

def K_rho_beta(p, beta):
    _aux_ = 0 + 1j*0
    
    for k in np.arange(- max_in_sum_2, max_in_sum_2 + 1):
        idx = find_nearest(radius, p)
        _aux_ += K_n_loaded[k + max_in_sum_2, idx] * np.exp(1j * k * beta)
    
    return _aux_

## Discretizamos

A continuación discretizamos a antena para calcular as excitacións usando o método de valor media dos vértices do cadrado.

In [ ]:
radio_vector *= a / np.max(radio_vector)

In [ ]:
I_mn = []
I_x = []
I_y = []

sign_x = np.array([1, 1, -1, -1])
sign_y = np.array([1, -1, 1, -1])

limit = (np.int(a / spacing_dipoles) + 0.5) * spacing_dipoles

for y in tqdm(np.arange(- limit, limit + 1, spacing_dipoles)):        
    for x in np.arange(- limit, limit + 1, spacing_dipoles):
        r_ij = np.sqrt(x**2 + y**2)
        beta_ij = np.arctan2(y, x)
        m = int(1800 + 10 * degrees(beta_ij))
        
        if r_ij <= radio_vector[m]:
            aux = 0 + 1j*0

            for i in range(4):
                new_x = x + sign_x[i] * spacing_dipoles * 0.5
                new_y = y + sign_y[i] * spacing_dipoles * 0.5

                r_ij = np.sqrt(new_x**2 + new_y**2)
                beta_ij = np.arctan2(new_y, new_x)
                aux += K_rho_beta(r_ij, beta_ij)

            I_mn.append(0.25 * aux)
            I_x.append(x)
            I_y.append(y)

I_mn /= np.max(I_mn)

# Como método de optimización eliminamos os elementos con excitación menor á establecida.

excitation_threeshold = 0.1
removable_elements = np.where(np.abs(I_mn) <= excitation_threeshold)[0]

aux_I_x = I_x.copy()
aux_I_y = I_y.copy()
I_mn = np.delete(I_mn, removable_elements)
I_x = np.delete(I_x, removable_elements)
I_y = np.delete(I_y, removable_elements)

In [ ]:
plt.plot(aux_I_x, aux_I_y, 'o')
plt.plot(I_x, I_y, 'o')
plt.xlim(-a, a)
plt.ylim(-a, a)
plt.savefig(f'Results/Images/Antenna_shape_{boundary}_{max_in_sum_1}_{max_in_sum_2}.eps', dpi = 300)

## Calculamos $F(\theta, \phi)$

In [ ]:
def f_subarray(u, v):
    if Subarray:
        psi_x = 2 * np.pi * spacing_subarray * u
        psi_y = 2 * np.pi * spacing_subarray * v
        return np.sin(0.5 * M * psi_x) * np.sin(0.5 * N * psi_y) / (np.sin(0.5 * psi_x) * np.sin(0.5 * psi_y))
    else:
        return 1

def f_element(u, v):
    if Element:
        return np.sin(0.5 * np.pi * np.sqrt(1 - u**2 - v**2)) * np.cos(2 * np.pi * 0.495 * u) / np.sqrt(1 - u**2)
    else:
        return 1

def F_theta_phi(u, v):
    F = 0 + 1j*0
    
    for k in range(len(I_mn)):
        exponent = 2 * np.pi * (I_x[k] * u + I_y[k] * v)
        F += I_mn[k] * np.exp(1j * exponent)
        
    return F * f_subarray(u, v) * f_element(u, v)

# Plotting the pattern

Creamos o enreixado.

In [ ]:
x = np.linspace(- 1, 1, 661, endpoint = True); x[np.where(x == 0)] = 1e-5
X, Y = np.meshgrid(x, x)
len_x = len(x)

out_arg = np.where(X**2 + Y**2 >= 1)

X[out_arg] = 0.5
Y[out_arg] = 0.5

Calculamos os valores do patrón, limpámolo e xeramos a imaxe.

In [ ]:
limit = -30

Z = np.abs(F_theta_phi(X, Y))
Z = 2 * pass_to_dB(Z)
clear_around(Z, limit)
Z = np.reshape(Z, (len_x, len_x))

for i in tqdm(range(len_x)):
    for j in range(len_x):
        x_0 = i * np.pi/len_x - np.pi/2
        y_0 = j * np.pi/len_x - np.pi/2
        
        if np.square(x_0*x_0 + y_0*y_0) > np.pi/2:
            Z[i, j] = limit

Xeramos a imaxe e gardamos resultados.

In [ ]:
np.savetxt(f'Results/Data/Data_F_{boundary}.txt', Z)

fig = plt.figure(1)
plt.imshow(Z) 
plt.title('$F(\Theta, \phi)$')
plt.savefig(f'Results/Images/Horizontal_F_{boundary}.eps', dpi = 300)

fig = plt.figure(2)
ax = plt.axes(projection = '3d')
X_plot, Y_plot = np.meshgrid(x, x)
ax.plot_surface(X_plot, Y_plot, Z, cmap = 'viridis')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.grid(False)
ax.set_visible(True)
ax.set_zlim(limit, 0)
plt.title('$F(\Theta, \phi)$')
ax.view_init(0, 90)
plt.savefig(f'Results/Images/3D_F_{boundary}.eps', dpi = 300)

# Plotting the excitations

rho = np.sqrt(X_plot**2 + Y_plot**2) * a
beta = np.arctan2(Y_plot, X_plot)

rho = np.reshape(rho, (len_x*len_x))
beta = np.reshape(beta, (len_x*len_x))

Z = np.zeros((len_x*len_x))

for i in tqdm(range(len_x * len_x)):
    Z[i] = np.abs(K_rho_beta(rho[i], beta[i]))

Z = np.reshape(Z, (len_x, len_x))
np.savetxt(f'Results/Data/Data_K_{boundary}.txt', Z)

fig = plt.figure(1)
plt.imshow(Z)
plt.title(r'$K(\rho, \beta)$')
plt.savefig(f'Results/Images/Horizontal_K_{boundary}.eps', dpi = 300)

fig = plt.figure(2)
ax = plt.axes(projection = '3d')
ax.plot_surface(X_plot, Y_plot, Z, cmap = 'viridis')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.grid(False)
ax.set_visible(True)
plt.title(r'$K(\rho, \beta)$')
plt.savefig(f'Results/Images/3D_K_{boundary}.eps', dpi = 300)
ax.view_init(50, 50)

## Ratio de rango dinámico

In [ ]:
def dynamic_range_ratio(array):
    array = np.abs(array)
    return np.max(array) / np.min(array)

print(f'O rango dinámico é de: {dynamic_range_ratio(I_mn)}')

In [ ]:
os.remove('Temporal_radial_vector.txt')
print(f'Total execution time is: {np.round(time.perf_counter() - t0, 2)} s')